In [20]:
# !pip install -q tensorflow-model-optimization pyyaml loguru model-profiler
# !pip install timm
# !pip install onnxruntime
# !pip install onnx
!pip install openvino

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 26.2 MB/s eta 0:00:00


In [2]:
!git clone -b dev https://github.com/SashaMogilevskii/model_compression

Cloning into 'model_compression'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 147 (delta 7), reused 21 (delta 2), pack-reused 118
Receiving objects: 100% (147/147), 78.38 MiB | 23.12 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [3]:
%cd model_compression/data

/content/model_compression/data


In [4]:
!gdown --id 18gCWw8IkqbFr9X0TCBM5LCpsU11Ga6tc
!unrar x -y data.rar

Streaming output truncated to the last 5000 lines.
Extracting  data/base_audio/rerswa1/XC471774.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC471955.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC471957.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC471959.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC472354.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC478273.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC479591.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC482315.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC485355.ogg                          73%  OK 
Extracting  data/base_audio/rerswa1/XC489638.ogg                          73%  OK 
Extracting  data/base_audio/r

In [5]:
%cd ../hw

/content/model_compression/hw


In [6]:
import sys
sys.path.append("../")

In [7]:
import sys
sys.path.append("../scr/utils")
import os

from copy import deepcopy
import time
import platform
import onnx
import onnxruntime as ort
import numpy as np
from tqdm import tqdm

import timm
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

from scr.utils.create_dataset import BirdDataset
from scr.utils.metrics import validation_epoch_end

In [8]:
# Информация о железе, на котором тестируется модель

processor_info = platform.processor()
print("Информация о процессоре:", processor_info)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print("Доступно GPU:", num_gpus)
    for i in range(num_gpus):
        gpu = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu}")
else:
    print("GPU недоступны на данной системе.")

Информация о процессоре: x86_64
GPU недоступны на данной системе.


In [66]:
def get_metric_score(model):
    predicted_labels_list = None
    true_labels_list = None
    metric = validation_epoch_end
    start_time = time.time()


    with torch.no_grad():
        for batch in tqdm(valid_loader):
            X_batch = batch[0].to(device)
            y_batch = batch[1].to(device)
            res = model.forward(X_batch)

            res = res.detach().sigmoid().cpu().numpy()
            y_batch_onehot = y_batch
            y_batch_onehot = y_batch_onehot.unsqueeze(1).detach().cpu().numpy()
            y_batch_onehot = y_batch_onehot.squeeze()

            if predicted_labels_list is None:
                predicted_labels_list = res
                true_labels_list = y_batch_onehot
            else:
                predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
                true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)

            del batch, res
        torch.cuda.empty_cache()

    end_time = time.time()


    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0
    avg_metric = metric(all_true_labels.reshape(100, -1), all_predicted_labels.reshape(100, -1))
    t = end_time - start_time

    # Мы понимаем, что ко времени работы модели добавляем время обработки батчей и добавления аугментаций к стартовым данным

    print("Наши метрики на нашей базовой модели:")
    print(f"Время работы модели на всем батче {t:.<2g} сек.")
    print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
    print("Метрики качества:")
    for m in avg_metric:
        print(f"metric {m} : {avg_metric[m]:.<5g}")

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

sigmoid_v = np.vectorize(sigmoid)

def get_metric_score_onnx(ort_session, loader, validation_epoch_end):
    predicted_labels_list = None
    true_labels_list = None
    start_time = time.time()

    for batch in tqdm(loader):
        X_batch = batch[0].numpy()
        y_batch = batch[1]

        # ort_inputs = {'x.1':  np.expand_dims(X_batch, 0)}
        ort_inputs = {'x.1':  X_batch}
        ort_outputs = ort_session.run(None, ort_inputs)

        res = sigmoid_v(ort_outputs[0]).reshape(-1, 1)

        y_batch_onehot = y_batch
        y_batch_onehot = y_batch_onehot.reshape(-1, 1)

        if predicted_labels_list is None:
            predicted_labels_list = res
            true_labels_list = y_batch_onehot
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)

    end_time = time.time()

    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0

    avg_metric = validation_epoch_end(all_true_labels.reshape(100, -1), all_predicted_labels.reshape(100, -1))
    t = end_time - start_time

    print("Наши метрики на нашей ONNX модели:")
    print(f"Время работы модели на всем батче {t:.2f} сек.")
    print(f"Время работы модели на одном сэмпле (AVG) {round(t / len(valid_loader), 3)} сек.")
    print("Метрики качества:")
    for m in avg_metric:
        print(f"metric {m} : {avg_metric[m]:.5f}")


def get_metric_score_openvino(compiled_model, loader, validation_epoch_end):
    predicted_labels_list = None
    true_labels_list = None
    start_time = time.time()

    for batch in tqdm(loader):
        X_batch = batch[0].numpy()
        y_batch = batch[1]

        outputs = compiled_model(X_batch)['1072']

        res = sigmoid_v(outputs[0]).reshape(-1, 1)

        y_batch_onehot = y_batch
        y_batch_onehot = y_batch_onehot.reshape(-1, 1)

        if predicted_labels_list is None:
            predicted_labels_list = res
            true_labels_list = y_batch_onehot
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)

    end_time = time.time()

    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0

    avg_metric = validation_epoch_end(all_true_labels.reshape(100, -1), all_predicted_labels.reshape(100, -1))
    t = end_time - start_time

    print("Наши метрики на нашей ONNX модели:")
    print(f"Время работы модели на всем батче {t:.2f} сек.")
    print(f"Время работы модели на одном сэмпле (AVG) {round(t / len(valid_loader), 3)} сек.")
    print("Метрики качества:")
    for m in avg_metric:
        print(f"metric {m} : {avg_metric[m]:.5f}")


In [10]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))


def get_onnx_model_size(onnx_model_path):
    # Получите размер файла ONNX модели
    model_size_bytes = os.path.getsize(onnx_model_path)

    # Переведите размер в мегабайты (MB)
    model_size_mb = model_size_bytes / (1024 ** 2)

    print(f'ONNX модель размером: {model_size_mb:.3f} MB')

def count_zero_weights(model):
    num_zero_weights = 0
    total_weights = 0

    for name, param in model.named_parameters():
        if 'weight' in name:
            num_zero_weights += torch.sum(param == 0).item()
            total_weights += param.numel()

    sparsity = num_zero_weights / total_weights
    print(f"Разреженность весов: {sparsity * 100:.2f}%")
    return sparsity

In [11]:
df = pd.read_csv("../data/data.csv")
df_test = df[df.fold == 3].sample(n=100, random_state=42).reset_index(drop=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_base = torch.load("../models/model_pytorch/model_tf_efficientnet_b0_last_version.pt", map_location=device).to(device)
dataset_test = BirdDataset(df=df_test, path_to_folder_with_audio="../data/data/")
valid_loader = DataLoader(dataset_test, batch_size=1)
model_base.eval()
device

device(type='cpu')

In [12]:
get_metric_score(model_base)

100%|██████████| 100/100 [00:47<00:00,  2.10it/s]

Наши метрики на нашей базовой модели:
Время работы модели на всем батче 47.5928 сек.
Время работы модели на одном сэмпле (AVG) 0.476 сек.
Метрики качества:
metric val_RMAP : 0.676704


In [13]:
batch_size = 1
x = torch.randn(batch_size, 3, 626, 256, requires_grad=True)
torch_out = model_base(x)

torch.onnx.export(model_base, x, "../models/model.onnx", opset_version=11)

In [14]:
onnx_model = onnx.load("../models/model.onnx")
onnx.checker.check_model(onnx_model)

In [15]:
ort_sess = ort.InferenceSession("../models/model.onnx")

input = next(iter(dataset_test))[0]
outputs = ort_sess.run(None, {'x.1': np.expand_dims(input.numpy(), 0)})

In [17]:
get_metric_score_onnx(ort_sess, valid_loader, validation_epoch_end)

100%|██████████| 100/100 [00:27<00:00,  3.62it/s]


Наши метрики на нашей ONNX модели:
Время работы модели на всем батче 27.68 сек.
Время работы модели на одном сэмпле (AVG) 0.277 сек.
Метрики качества:
metric val_RMAP : 0.68282


In [18]:
get_onnx_model_size("../models/model.onnx")

ONNX модель размером: 16.617 MB


In [64]:
import openvino as ov

ov_model = ov.convert_model('../models/model.onnx')
ov.runtime.serialize(ov_model, '../models/openvino_model/model.xml')

compiled_model = ov.compile_model(ov_model)

In [67]:
get_metric_score_openvino(compiled_model, valid_loader, validation_epoch_end)

100%|██████████| 100/100 [00:24<00:00,  4.00it/s]


Наши метрики на нашей ONNX модели:
Время работы модели на всем батче 24.98 сек.
Время работы модели на одном сэмпле (AVG) 0.25 сек.
Метрики качества:
metric val_RMAP : 0.70746


In [68]:
file_size_bytes = os.path.getsize('../models/openvino_model/model.bin')
file_size_mb = round(file_size_bytes / (1024 * 1024), 3) # Перевод из байтов в мегабайты

print(f"Размер compressed Openvino модели: {file_size_mb} MB")

Размер compressed Openvino модели: 16.498 MB
